In [8]:
import time
import os

import requests
import pandas as pd

import pickle
from pymongo import MongoClient

from config import genius_access_token

### Connecting to MongoDB

In [2]:
client = MongoClient()

In [3]:
db = client.genius

In [4]:
db.list_collection_names()

['song_lyrics', 'artist_ids', 'song_ids']

## Get Artist IDs

In [7]:
cursor = db.song_ids.find({}, {'_id':0, 'id':1})
song_ids = pd.DataFrame(list(cursor))
song_ids = song_ids.id.values.tolist()
song_ids[:5]

[987434, 1009076, 1551187, 1745373, 1073422]

## Test Query

In [ ]:
access_token = genius_access_token
headers={
         "Accept": "application/json",
         "Content-Type": "application/json",
         "Authorization": "Bearer " + access_token
        }

In [ ]:
url = "http://api.genius.com/artists/{}/songs?page={}".format(artist_id, page_num)
resp = requests.get(url, headers=headers)
resp.raise_for_status()
json = resp.json()

## Write to mongoDB

In [15]:
def get_json(headers, artist_id, page_num):
    url = "http://api.genius.com/artists/{}/songs?page={}".format(artist_id, page_num)
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    json = resp.json()
    return json

In [20]:
def save_song_info_to_mongoDB(json, artist_id):
    song_list = json['response']['songs']
    
    for song in song_list:
        song_info = {}
        song_info['artist_id'] = artist_id
        features = ['title','id','url']
        
        for feature in features:
            song_info[feature] = song[feature]
            
        db.song_ids.insert_one(song_info)

In [22]:
def get_artist_ids(artist_ids, access_token, headers):
    
    for artist_id in artist_ids:
        page_num = 1
        json = get_json(headers, artist_id, page_num)
        save_song_info_to_mongoDB(json, artist_id)
        
        while json['response']['next_page']:
            page_num += 1
            json = get_json(headers, artist_id, page_num)
            save_song_info_to_mongoDB(json, artist_id)

In [18]:
access_token = genius_access_token
headers={
         "Accept": "application/json",
         "Content-Type": "application/json",
         "Authorization": "Bearer " + access_token
        }

In [23]:
get_artist_ids(artist_ids, access_token, headers)